This is a very brief demo to show how one could go from an environment with just a Python 3 runtime to pyhf installed and being useful.

## Installation

To take advantage of the great tools that `uproot` provides us let's install `pyhf` with the [`xmlio` options](https://diana-hep.org/pyhf/examples/notebooks/XML_ImportExport.html)

```
$ pip3 install pyhf[xmlio]
```

but this is Binder! so I've already done that for you. :)

## Running with CLI

Now that we have `pyhf` installed let's ask what the CLI can do for us

In [1]:
! pyhf --help

Usage: pyhf [OPTIONS] COMMAND [ARGS]...

Options:
  --version   Show the version and exit.
  -h, --help  Show this message and exit.

Commands:
  cls
  inspect
  json2xml
  xml2json  Entrypoint XML: The top-level XML file for the PDF definition.


and then let's use `pyhf inspect` to check to see if our `example.json` is in a correct form for the schema

In [2]:
! pyhf inspect example.json

          Summary       
    ------------------  
       channels  1
        samples  2
     parameters  2
      modifiers  2

       channels  nbins
     ----------  -----
  singlechannel    2  

        samples
     ----------
     background
         signal

     parameters  constraint              modifiers
     ----------  ----------              ----------
             mu  unconstrained           normfactor
uncorr_bkguncrt  constrained_by_poisson  shapesys

    measurement           poi            parameters
     ----------        ----------        ----------
(*) Measurement            mu            (none)



finally use `pyhf cls` to get the observed CL_s and the expected CL_s band values (the "[Brazil Band](https://arxiv.org/pdf/1306.3117.pdf#page=102)")

In [3]:
! pyhf cls example.json

{
    "CLs_exp": [
        0.07807427911686152,
        0.17472571775474582,
        0.35998495263681274,
        0.6343568235898907,
        0.8809947004472013
    ],
    "CLs_obs": 0.3599845631401913
}


### Expanding the CLI with other tools

Additionally, you can also use [JSON Patch](https://tools.ietf.org/html/rfc6902) to patch a new signal in and update the likelihood (think reinterpretation).

In [4]:
! pyhf cls example.json --patch new_signal.json

{
    "CLs_exp": [
        0.14695763778146098,
        0.2738178123017487,
        0.47642647066827803,
        0.7289555883715226,
        0.922094011644031
    ],
    "CLs_obs": 0.4764263982925686
}


and as the return format from `pyhf cls` is JSON you can also use [`jq`](https://stedolan.github.io/jq/) to filter results

In [5]:
! pyhf cls example.json --patch new_signal.json | jq .CLs_obs

0.4764263982925686


and of course given the way the CLI is setup you can pipe in the JSON which means that you can grab remotely hosted files (think HEPData). Here we `curl` the `example.json` down from GitHub

In [6]:
! curl -sL https://raw.githubusercontent.com/matthewfeickert/talk-PyHEP-2019/master/notebooks/example.json \
    | pyhf cls --patch new_signal.json

{
    "CLs_exp": [
        0.14695763778146098,
        0.2738178123017487,
        0.47642647066827803,
        0.7289555883715226,
        0.922094011644031
    ],
    "CLs_obs": 0.4764263982925686
}


Those URLs are hard to read (Jupyter shell magics are only so magic), so the following is happening:

```
curl -sL https://web.end.point/model.json \
  | pyhf cls --patch <(curl -sL https://web.end.point/new_signal.json)
```